# Data engineering

***

## Load and prepare data

To download and prepare the election data, you will use ArcPy, the ArcGIS API for Python, and a Pandas dataframe. First, you will import these modules to use them. Then, you will create a variable for the United States county election data and use this variable to read the data into a Pandas dataframe.

##### Import needed modules

In [81]:
import arcgis
import arcpy
import os
import pandas as pd

This notebook describes the process to download and prepare United States presidential election data. You will address missing values, reformat data types, and restructure the format of a table.

##### Read data into Python

In [82]:
table_csv_path = "countypres2016.csv"

In [83]:
data_df = pd.read_csv(table_csv_path)

In [84]:
data_df.head()

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2016,Alabama,AL,Autauga,1001.0,President,Hillary Clinton,democrat,5936.0,24973,20190722
1,2016,Alabama,AL,Autauga,1001.0,President,Donald Trump,republican,18172.0,24973,20190722
2,2016,Alabama,AL,Autauga,1001.0,President,Other,NaN,865.0,24973,20190722
3,2016,Alabama,AL,Baldwin,1003.0,President,Hillary Clinton,democrat,18458.0,95215,20190722
4,2016,Alabama,AL,Baldwin,1003.0,President,Donald Trump,republican,72883.0,95215,20190722


***

## Handle missing data 

The election data includes a records that are missing data in the FIPS field. This missing data is referred to as null values. You will identify how many rows have null values and create a new dataframe that does not include them.
![Null Values](img/null_values.gif "Null Values")

In [85]:
# Perform a query on the dataframe using the loc function and the necessary field name.
data_df.loc[data_df['FIPS'].isnull()]  # We can use the isnull function built in to Pandas to find the records with null FIPS.

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
9462,2016,Connecticut,NaN,Statewide writein,NaN,President,Hillary Clinton,democrat,NaN,5056,20190722
9463,2016,Maine,NaN,Maine UOCAVA,NaN,President,Hillary Clinton,democrat,3017.0,5056,20190722
9464,2016,Alaska,NaN,District 99,NaN,President,Hillary Clinton,democrat,274.0,5056,20190722
9465,2016,Rhode Island,NaN,Federal Precinct,NaN,President,Hillary Clinton,democrat,637.0,5056,20190722
9466,2016,Connecticut,NaN,Statewide writein,NaN,President,Donald Trump,republican,NaN,5056,20190722
9467,2016,Maine,NaN,Maine UOCAVA,NaN,President,Donald Trump,republican,648.0,5056,20190722
9468,2016,Alaska,NaN,District 99,NaN,President,Donald Trump,republican,40.0,5056,20190722
9469,2016,Rhode Island,NaN,Federal Precinct,NaN,President,Donald Trump,republican,53.0,5056,20190722
9470,2016,Connecticut,NaN,Statewide writein,NaN,President,Other,NaN,NaN,5056,20190722
9471,2016,Maine,NaN,Maine UOCAVA,NaN,President,Other,NaN,321.0,5056,20190722


In [86]:
# Determine how many rows are in the table
rowcount = data_df.shape[0]

# Determine how many rows have null FIPS 
null_fips_rowcount = data_df.loc[data_df['FIPS'].isnull()].shape[0]

# Calculate how much of the data this represents as a percentage
percentage_null_fips = round((null_fips_rowcount / rowcount) * 100, 2)

# Use a print statement to report this information
print("There were "+str(null_fips_rowcount)+" records with null FIPS values in the data.\nThis amounts to " +str(percentage_null_fips)+"% of the available data.")

There were 12 records with null FIPS values in the data.
This amounts to 0.13% of the available data.


In [87]:
# Use the notnull function and the loc function to create a new dataframe without null FIPS records
data_df = data_df.loc[data_df['FIPS'].notnull()]

In [88]:
data_df['FIPS']

0        1001.0
1        1001.0
2        1001.0
3        1003.0
4        1003.0
         ...   
9457    56043.0
9458    56043.0
9459    56045.0
9460    56045.0
9461    56045.0
Name: FIPS, Length: 9462, dtype: float64

In [89]:
data_df = data_df.astype({'FIPS': 'str'})

In [96]:
data_df.FIPS = data_df.FIPS.str[:-2]

***

## Explore and handle data types

In reviewing your data, you notice that the FIPS field is considered a numeric field instead of a string. As a result, leading zeroes in the FIPS values have been removed. The resulting FIPS values only have four characters instead of five. You will determine how many records are missing leading zeroes and add, or append, the missing zero.
![fix_truncated_zeroes](img/trunc_zeroes.gif "Fix Truncated Zeroes")

In [97]:
# Get the first five records of the table
data_df.head()

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2016,Alabama,AL,Autauga,1001,President,Hillary Clinton,democrat,5936.0,24973,20190722
1,2016,Alabama,AL,Autauga,1001,President,Donald Trump,republican,18172.0,24973,20190722
2,2016,Alabama,AL,Autauga,1001,President,Other,NaN,865.0,24973,20190722
3,2016,Alabama,AL,Baldwin,1003,President,Hillary Clinton,democrat,18458.0,95215,20190722
4,2016,Alabama,AL,Baldwin,1003,President,Donald Trump,republican,72883.0,95215,20190722


In [98]:
# Check how many records have a FIPS value with four characters
trunc_df = data_df.loc[data_df.astype({'FIPS': 'str'}).FIPS.str.len() == 4]
trunc_data_per = (trunc_df.shape[0] / data_df.shape[0])*100

# Use another print statement (using the f format key) to report this information
print(f"{round(trunc_data_per, 2)}% of data ({trunc_df.shape[0]} rows) has truncated FIPS values.")

10.37% of data (981 rows) has truncated FIPS values.


In [47]:
data_df.astype({'FIPS': 'str'}).FIPS.str.len()

0       6
1       6
2       6
3       6
4       6
       ..
9457    7
9458    7
9459    7
9460    7
9461    7
Name: FIPS, Length: 9462, dtype: int64

The following cell creates a function in python that adds a leading zero to the FIPS value if it only has four characters.  

In [99]:
# Define a helper function to fix truncated zeros, with one parameter: the value to be processed
def fix_trunc_zeros(val):
    # Use an if statement to check if there are four characters in the string representation of the value
    if len(str(val)) == 4:
        # If this is the case, return the value with an appended "0" in the front
        return "0"+str(val)
    # Otherwise...
    else:
        # Return the value itself
        return str(val)

In [101]:
# Test helper function with truncated value
fix_trunc_zeros(7042)  # You should see an appended zero: "07042"

'07042'

In [102]:
# Run helper function on the FIPS field using the apply and lambda method 
data_df['FIPS'] = data_df['FIPS'].apply(lambda x: fix_trunc_zeros(x))

# Print information on the operation performed, and show the first few records to confirm it worked
print(f"{round(trunc_data_per, 2)}% of data ({trunc_df.shape[0]} rows) had truncated FIPS IDs corrected.")
data_df.head()

10.37% of data (981 rows) had truncated FIPS IDs corrected.


,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2016,Alabama,AL,Autauga,01001,President,Hillary Clinton,democrat,5936.0,24973,20190722
1,2016,Alabama,AL,Autauga,01001,President,Donald Trump,republican,18172.0,24973,20190722
2,2016,Alabama,AL,Autauga,01001,President,Other,NaN,865.0,24973,20190722
3,2016,Alabama,AL,Baldwin,01003,President,Hillary Clinton,democrat,18458.0,95215,20190722
4,2016,Alabama,AL,Baldwin,01003,President,Donald Trump,republican,72883.0,95215,20190722


***

## Reformat the table structure

Currently, each record in the table corresponds to a candidate and their votes in a county. You need to reformat the table so that each record corresponds to each county, with fields showing the votes for different candidates in that election year. 
It is possible to do this using the [Pivot Table geoprocessing tool](https://pro.arcgis.com/en/pro-app/tool-reference/data-management/pivot-table.htm) or Excel pivot tables, but Python may make it easier to automate and share.
The animation below illustrates the steps in restructuring the table:
1. Set a few fields aside, "locking" them from the table pivot. 
2. Pivot the table using the remaining fields.
3. Rename the pivoted fields to designate each party. 
4. Bring the locked fields back to the table. 
The following code cell performs these steps.
![reformat_table](img/reformat_table.gif "Reformat Table")


In [103]:
# Set an index using mulitple fields, which "locks" these fields before the table pivots
# Use the built-in groupby function for the FIPS and year fields, which you use to group the data by candidate
# Use unstack to perform the table pivot, which will rotate the table and turn rows into columns
df_out = data_df.set_index(['FIPS', 
                            'year', 
                            'county', 
                            'state', 
                            'state_po', 
                            'office', 
                            data_df.groupby(['FIPS', 'year']).cumcount()+1]).unstack()

# Use the indexes for the columns to set column names (Ex: candidate_1, candidate_2, votes_1, votes_2, etc.)
df_out.columns = df_out.columns.map('{0[0]}_{0[1]}'.format)

# Rename columns 
df_out = df_out.rename(columns={"candidate_1": "candidate_dem",
                                "candidatevotes_1": "votes_dem",
                                "candidate_2": "candidate_gop",
                                "candidatevotes_2": "votes_gop",
                                "totalvotes_1": "votes_total",
                                "state_po": "state_abbrev"
                                })

# Keep only the necessary columns
df_out = df_out[["candidate_dem", "votes_dem",
                 "candidate_gop", "votes_gop",
                 "votes_total"]]

# Remove the multiindex since we no longer need these fields to be "locked" for the pivot
df_out.reset_index(inplace=True)

# Print out the first few records to confirm everything worked
df_out.head()

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,votes_total
0,01001,2016,Autauga,Alabama,AL,President,Hillary Clinton,5936.0,Donald Trump,18172.0,24973
1,01003,2016,Baldwin,Alabama,AL,President,Hillary Clinton,18458.0,Donald Trump,72883.0,95215
2,01005,2016,Barbour,Alabama,AL,President,Hillary Clinton,4871.0,Donald Trump,5454.0,10469
3,01007,2016,Bibb,Alabama,AL,President,Hillary Clinton,1874.0,Donald Trump,6738.0,8819
4,01009,2016,Blount,Alabama,AL,President,Hillary Clinton,2156.0,Donald Trump,22859.0,25588


Pandas has three powerful capabilities that helped you perform this operation: 
- The ability to set an index using multiple fields, which acts as our "locking" mechanism. 
- The ability to unstack (or pivot) a table.
- The ability to perform an operation using a "groupby" function.

## Calculate additional columns

You will use the values from the updated table to add additional columns of information, such as the number of votes for a non major party, the percentage of voters for each party, and so on. Each column is referred to as an attribute of the dataset.

##### Calculate an attribute for the total votes for non major party

In [104]:
# Calculate votes that did not choose the Democratic or Republican party
df_out['votes_other'] = df_out['votes_total'] - (df_out['votes_dem'] + df_out['votes_gop'])
df_out.head()

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,votes_total,votes_other
0,01001,2016,Autauga,Alabama,AL,President,Hillary Clinton,5936.0,Donald Trump,18172.0,24973,865.0
1,01003,2016,Baldwin,Alabama,AL,President,Hillary Clinton,18458.0,Donald Trump,72883.0,95215,3874.0
2,01005,2016,Barbour,Alabama,AL,President,Hillary Clinton,4871.0,Donald Trump,5454.0,10469,144.0
3,01007,2016,Bibb,Alabama,AL,President,Hillary Clinton,1874.0,Donald Trump,6738.0,8819,207.0
4,01009,2016,Blount,Alabama,AL,President,Hillary Clinton,2156.0,Donald Trump,22859.0,25588,573.0


##### Calculate additional attributes

In [105]:
# Calculate voter share attributes
df_out['voter_share_major_party'] = (df_out['votes_dem'] + df_out['votes_gop']) / df_out['votes_total']
df_out['voter_share_dem'] = df_out['votes_dem'] / df_out['votes_total']
df_out['voter_share_gop'] = df_out['votes_gop'] / df_out['votes_total']
df_out['voter_share_other'] = df_out['votes_other'] / df_out['votes_total']

# Calculate raw difference attributes
df_out['rawdiff_dem_vs_gop'] = df_out['votes_dem'] - df_out['votes_gop']
df_out['rawdiff_gop_vs_dem'] = df_out['votes_gop'] - df_out['votes_dem']
df_out['rawdiff_dem_vs_other'] = df_out['votes_dem'] - df_out['votes_other']
df_out['rawdiff_gop_vs_other'] = df_out['votes_gop'] - df_out['votes_other']
df_out['rawdiff_other_vs_dem'] = df_out['votes_other'] - df_out['votes_dem']
df_out['rawdiff_other_vs_gop'] = df_out['votes_other'] - df_out['votes_gop']

# Calculate percent difference attributes
df_out['pctdiff_dem_vs_gop'] = (df_out['votes_dem'] - df_out['votes_gop']) / df_out['votes_total']
df_out['pctdiff_gop_vs_dem'] = (df_out['votes_gop'] - df_out['votes_dem']) / df_out['votes_total']
df_out['pctdiff_dem_vs_other'] = (df_out['votes_dem'] - df_out['votes_other']) / df_out['votes_total']
df_out['pctdiff_gop_vs_other'] = (df_out['votes_gop'] - df_out['votes_other']) / df_out['votes_total']
df_out['pctdiff_other_vs_dem'] = (df_out['votes_other'] - df_out['votes_dem']) / df_out['votes_total']
df_out['pctdiff_other_vs_gop'] = (df_out['votes_other'] - df_out['votes_gop']) / df_out['votes_total']

df_out.head()

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,votes_total,votes_other,voter_share_major_party,voter_share_dem,voter_share_gop,voter_share_other,rawdiff_dem_vs_gop,rawdiff_gop_vs_dem,rawdiff_dem_vs_other,rawdiff_gop_vs_other,rawdiff_other_vs_dem,rawdiff_other_vs_gop,pctdiff_dem_vs_gop,pctdiff_gop_vs_dem,pctdiff_dem_vs_other,pctdiff_gop_vs_other,pctdiff_other_vs_dem,pctdiff_other_vs_gop
0,01001,2016,Autauga,Alabama,AL,President,Hillary Clinton,5936.0,Donald Trump,18172.0,24973,865.0,0.965363,0.237697,0.727666,0.034637,-12236.0,12236.0,5071.0,17307.0,-5071.0,-17307.0,-0.489969,0.489969,0.203059,0.693028,-0.203059,-0.693028
1,01003,2016,Baldwin,Alabama,AL,President,Hillary Clinton,18458.0,Donald Trump,72883.0,95215,3874.0,0.959313,0.193856,0.765457,0.040687,-54425.0,54425.0,14584.0,69009.0,-14584.0,-69009.0,-0.571601,0.571601,0.153169,0.724770,-0.153169,-0.724770
2,01005,2016,Barbour,Alabama,AL,President,Hillary Clinton,4871.0,Donald Trump,5454.0,10469,144.0,0.986245,0.465278,0.520967,0.013755,-583.0,583.0,4727.0,5310.0,-4727.0,-5310.0,-0.055688,0.055688,0.451524,0.507212,-0.451524,-0.507212
3,01007,2016,Bibb,Alabama,AL,President,Hillary Clinton,1874.0,Donald Trump,6738.0,8819,207.0,0.976528,0.212496,0.764032,0.023472,-4864.0,4864.0,1667.0,6531.0,-1667.0,-6531.0,-0.551536,0.551536,0.189024,0.740560,-0.189024,-0.740560
4,01009,2016,Blount,Alabama,AL,President,Hillary Clinton,2156.0,Donald Trump,22859.0,25588,573.0,0.977607,0.084258,0.893348,0.022393,-20703.0,20703.0,1583.0,22286.0,-1583.0,-22286.0,-0.809090,0.809090,0.061865,0.870955,-0.061865,-0.870955


***

## Geoenable the data

You will eventually use this data in a spatial analysis. This means that the data needs to include location information to determine where the data is located on a map. You will geoenable the data, or add location to the data, using existing geoenabled county data.

##### Define the ArcGIS Pro project, database, and existing geoenabled data

In [106]:
# Create variables that represent the ArcGIS Pro project and map
aprx = arcpy.mp.ArcGISProject("CURRENT")
mp = aprx.listMaps('Data Engineering')[0]

# Create a variable that represents the default file geodatabase
fgdb = r"Data Engineering and Visualization.gdb"
aprx.defaultGeodatabase = fgdb
arcpy.env.workspace = fgdb

There are various resources that you can use to find geoenabled data. [ArcGIS Living Atlas of the World](https://livingatlas.arcgis.com) is an authoritative source provided by Esri. Each record in your election data represents information for a county, so you will use a Living Atlas dataset that represents county geometry. This dataset has been downloaded and added to your project.

In [107]:
# Create a variable that represents the county geometry dataset
counties_fc_name = "Counties_2016_VotingAgePopulation"
counties_fc = os.path.join(fgdb, counties_fc_name)

**Note: Executing the following cell may take a few minutes.**

In [108]:
# Load the dataset into a spatially-enabled dataframe
counties_df = pd.DataFrame.spatial.from_featureclass(counties_fc)
counties_df.head()

,OBJECTID,COUNTYNS,GEOID,NAMELSAD,GEONAME,GEOID_1,LNNUMBER,Total_tot_est,Total_adu_est,Total_cit_est,Total_cvap_est,nHispLat_tot_est,nHispLat_adu_est,nHispLat_cit_est,nHispLat_cvap_est,AIAN_tot_est,AIAN_adu_est,AIAN_cit_est,AIAN_cvap_est,Asian_tot_est,Asian_adu_est,Asian_cit_est,Asian_cvap_est,BAA_tot_est,BAA_adu_est,BAA_cit_est,BAA_cvap_est,NHOPI_tot_est,NHOPI_adu_est,NHOPI_cit_est,NHOPI_cvap_est,White_tot_est,White_adu_est,White_cit_est,White_cvap_est,AIAN_W_tot_est,AIAN_W_adu_est,AIAN_W_cit_est,AIAN_W_cvap_est,Asian_W_tot_est,Asian_W_adu_est,Asian_W_cit_est,Asian_W_cvap_est,BAA_W_tot_est,BAA_W_adu_est,BAA_W_cit_est,BAA_W_cvap_est,AIAN_BAA_tot_est,AIAN_BAA_adu_est,AIAN_BAA_cit_est,AIAN_BAA_cvap_est,Rem_TOM_tot_est,Rem_TOM_adu_est,Rem_TOM_cit_est,Rem_TOM_cvap_est,HispLat_tot_est,HispLat_adu_est,HispLat_cit_est,HispLat_cvap_est,Shape__Area,Shape__Length,SHAPE
0,1,00161526,01001,Autauga County,"Autauga County, Alabama",05000US01001,13,55050,41195,54510,40690,53635,40290,53325,40015,225,125,225,125,485,390,340,245,10115,7470,10115,7470,4,4,4,4,41795,31835,41635,31705,230,220,230,220,270,140,270,140,420,50,420,50,0,0,0,0,90,55,90,55,1415,905,1185,675,2.208654e+09,2.498864e+05,"{'rings': [[[-9619465, 3856529.0001000017], [-..."
1,2,00161527,01003,Baldwin County,"Baldwin County, Alabama",05000US01003,13,199510,155240,195655,151770,190800,149730,188875,148020,1230,1125,1200,1095,1785,1195,1065,615,18500,13570,18245,13315,0,0,0,0,166275,132280,165455,131535,1275,795,1275,795,590,395,500,305,1035,260,1035,260,55,55,55,55,50,50,45,45,8710,5510,6780,3750,5.671048e+09,1.655940e+06,"{'rings': [[[-9746859, 3539643.0001000017], [-..."
2,3,00161528,01005,Barbour County,"Barbour County, Alabama",05000US01005,13,26615,20880,26085,20375,25465,20180,25405,20120,45,45,45,45,120,45,110,40,12765,9650,12745,9635,0,0,0,0,12390,10340,12360,10310,65,65,65,65,25,0,25,0,45,20,45,15,4,4,4,4,4,4,4,4,1145,700,680,250,3.257902e+09,3.208964e+05,"{'rings': [[[-9468394, 3771591.0001000017], [-..."
3,4,00161529,01007,Bibb County,"Bibb County, Alabama",05000US01007,13,22570,17815,22345,17590,22070,17440,22050,17415,80,45,80,45,15,15,0,0,4790,3875,4790,3875,0,0,0,0,16875,13255,16870,13250,65,65,65,65,0,0,0,0,230,170,230,170,0,0,0,0,15,15,15,15,500,380,295,175,2.311999e+09,2.279184e+05,"{'rings': [[[-9692114, 3928124.0001000017], [-..."
4,5,00161530,01009,Blount County,"Blount County, Alabama",05000US01009,13,57705,44105,55925,42430,52670,41085,52450,40885,170,155,170,155,90,80,75,65,905,675,895,675,0,0,0,0,50710,39710,50580,39580,450,360,450,360,105,75,55,25,175,25,175,25,0,0,0,0,60,4,45,4,5035,3015,3475,1550,2.456909e+09,2.926429e+05,"{'rings': [[[-9623907, 4063676.0001000017], [-..."


The county geometry dataset includes various attributes. You will simplify the dataframe to only include the attributes that you need. The Total_cvap_est attribute represents the total population in each county that are of voting age for the year 2016.

In [109]:
# Modify the dataframe to only include the attributes that are needed
counties_df = counties_df[['OBJECTID', 'GEOID', 'GEONAME',
                           'Total_cvap_est',
                           'SHAPE', 'Shape__Area', 'Shape__Length']]
counties_df.head()

,OBJECTID,GEOID,GEONAME,Total_cvap_est,SHAPE,Shape__Area,Shape__Length
0,1,01001,"Autauga County, Alabama",40690,"{'rings': [[[-9619465, 3856529.0001000017], [-...",2.208654e+09,2.498864e+05
1,2,01003,"Baldwin County, Alabama",151770,"{'rings': [[[-9746859, 3539643.0001000017], [-...",5.671048e+09,1.655940e+06
2,3,01005,"Barbour County, Alabama",20375,"{'rings': [[[-9468394, 3771591.0001000017], [-...",3.257902e+09,3.208964e+05
3,4,01007,"Bibb County, Alabama",17590,"{'rings': [[[-9692114, 3928124.0001000017], [-...",2.311999e+09,2.279184e+05
4,5,01009,"Blount County, Alabama",42430,"{'rings': [[[-9623907, 4063676.0001000017], [-...",2.456909e+09,2.926429e+05


***

## Join the data

You have a dataframe with election data ('df_out') and a spatially-enabled dataframe of the county geometry data ('counties_df'). You will merge these datasets into one. 

In [110]:
# Join the election dataframe with the county geometry dataframe
geo_df = pd.merge(df_out, counties_df, left_on='FIPS', right_on="GEOID", how='left')

# Visualize the merged data
geo_df.head()

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,votes_total,votes_other,voter_share_major_party,voter_share_dem,voter_share_gop,voter_share_other,rawdiff_dem_vs_gop,rawdiff_gop_vs_dem,rawdiff_dem_vs_other,rawdiff_gop_vs_other,rawdiff_other_vs_dem,rawdiff_other_vs_gop,pctdiff_dem_vs_gop,pctdiff_gop_vs_dem,pctdiff_dem_vs_other,pctdiff_gop_vs_other,pctdiff_other_vs_dem,pctdiff_other_vs_gop,OBJECTID,GEOID,GEONAME,Total_cvap_est,SHAPE,Shape__Area,Shape__Length
0,01001,2016,Autauga,Alabama,AL,President,Hillary Clinton,5936.0,Donald Trump,18172.0,24973,865.0,0.965363,0.237697,0.727666,0.034637,-12236.0,12236.0,5071.0,17307.0,-5071.0,-17307.0,-0.489969,0.489969,0.203059,0.693028,-0.203059,-0.693028,1.0,01001,"Autauga County, Alabama",40690.0,"{'rings': [[[-9619465, 3856529.0001000017], [-...",2.208654e+09,2.498864e+05
1,01003,2016,Baldwin,Alabama,AL,President,Hillary Clinton,18458.0,Donald Trump,72883.0,95215,3874.0,0.959313,0.193856,0.765457,0.040687,-54425.0,54425.0,14584.0,69009.0,-14584.0,-69009.0,-0.571601,0.571601,0.153169,0.724770,-0.153169,-0.724770,2.0,01003,"Baldwin County, Alabama",151770.0,"{'rings': [[[-9746859, 3539643.0001000017], [-...",5.671048e+09,1.655940e+06
2,01005,2016,Barbour,Alabama,AL,President,Hillary Clinton,4871.0,Donald Trump,5454.0,10469,144.0,0.986245,0.465278,0.520967,0.013755,-583.0,583.0,4727.0,5310.0,-4727.0,-5310.0,-0.055688,0.055688,0.451524,0.507212,-0.451524,-0.507212,3.0,01005,"Barbour County, Alabama",20375.0,"{'rings': [[[-9468394, 3771591.0001000017], [-...",3.257902e+09,3.208964e+05
3,01007,2016,Bibb,Alabama,AL,President,Hillary Clinton,1874.0,Donald Trump,6738.0,8819,207.0,0.976528,0.212496,0.764032,0.023472,-4864.0,4864.0,1667.0,6531.0,-1667.0,-6531.0,-0.551536,0.551536,0.189024,0.740560,-0.189024,-0.740560,4.0,01007,"Bibb County, Alabama",17590.0,"{'rings': [[[-9692114, 3928124.0001000017], [-...",2.311999e+09,2.279184e+05
4,01009,2016,Blount,Alabama,AL,President,Hillary Clinton,2156.0,Donald Trump,22859.0,25588,573.0,0.977607,0.084258,0.893348,0.022393,-20703.0,20703.0,1583.0,22286.0,-1583.0,-22286.0,-0.809090,0.809090,0.061865,0.870955,-0.061865,-0.870955,5.0,01009,"Blount County, Alabama",42430.0,"{'rings': [[[-9623907, 4063676.0001000017], [-...",2.456909e+09,2.926429e+05


The resulting dataframe includes the attributes from your election data and the specified attributes from the county geometry data. The SHAPE field represents the county geometry and is used to locate each record, or feature, on the map.

***

## Query and calculate attributes

Because you have the voting age population for 2016, you can now calculate the average voter participation (voter turnout) for 2016. The dataframe includes records from 2010-2016 but only has voting age population for 2016. You will need to create a subset dataframe for 2016 before calculating the voter turnout.

In [111]:
# Create a copy of the data, and perform a query
data_2016_df = geo_df.copy()
data_2016_df.query("year == '2016'", inplace=True)
data_2016_df.head()

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,votes_total,votes_other,voter_share_major_party,voter_share_dem,voter_share_gop,voter_share_other,rawdiff_dem_vs_gop,rawdiff_gop_vs_dem,rawdiff_dem_vs_other,rawdiff_gop_vs_other,rawdiff_other_vs_dem,rawdiff_other_vs_gop,pctdiff_dem_vs_gop,pctdiff_gop_vs_dem,pctdiff_dem_vs_other,pctdiff_gop_vs_other,pctdiff_other_vs_dem,pctdiff_other_vs_gop,OBJECTID,GEOID,GEONAME,Total_cvap_est,SHAPE,Shape__Area,Shape__Length
0,01001,2016,Autauga,Alabama,AL,President,Hillary Clinton,5936.0,Donald Trump,18172.0,24973,865.0,0.965363,0.237697,0.727666,0.034637,-12236.0,12236.0,5071.0,17307.0,-5071.0,-17307.0,-0.489969,0.489969,0.203059,0.693028,-0.203059,-0.693028,1.0,01001,"Autauga County, Alabama",40690.0,"{'rings': [[[-9619465, 3856529.0001000017], [-...",2.208654e+09,2.498864e+05
1,01003,2016,Baldwin,Alabama,AL,President,Hillary Clinton,18458.0,Donald Trump,72883.0,95215,3874.0,0.959313,0.193856,0.765457,0.040687,-54425.0,54425.0,14584.0,69009.0,-14584.0,-69009.0,-0.571601,0.571601,0.153169,0.724770,-0.153169,-0.724770,2.0,01003,"Baldwin County, Alabama",151770.0,"{'rings': [[[-9746859, 3539643.0001000017], [-...",5.671048e+09,1.655940e+06
2,01005,2016,Barbour,Alabama,AL,President,Hillary Clinton,4871.0,Donald Trump,5454.0,10469,144.0,0.986245,0.465278,0.520967,0.013755,-583.0,583.0,4727.0,5310.0,-4727.0,-5310.0,-0.055688,0.055688,0.451524,0.507212,-0.451524,-0.507212,3.0,01005,"Barbour County, Alabama",20375.0,"{'rings': [[[-9468394, 3771591.0001000017], [-...",3.257902e+09,3.208964e+05
3,01007,2016,Bibb,Alabama,AL,President,Hillary Clinton,1874.0,Donald Trump,6738.0,8819,207.0,0.976528,0.212496,0.764032,0.023472,-4864.0,4864.0,1667.0,6531.0,-1667.0,-6531.0,-0.551536,0.551536,0.189024,0.740560,-0.189024,-0.740560,4.0,01007,"Bibb County, Alabama",17590.0,"{'rings': [[[-9692114, 3928124.0001000017], [-...",2.311999e+09,2.279184e+05
4,01009,2016,Blount,Alabama,AL,President,Hillary Clinton,2156.0,Donald Trump,22859.0,25588,573.0,0.977607,0.084258,0.893348,0.022393,-20703.0,20703.0,1583.0,22286.0,-1583.0,-22286.0,-0.809090,0.809090,0.061865,0.870955,-0.061865,-0.870955,5.0,01009,"Blount County, Alabama",42430.0,"{'rings': [[[-9623907, 4063676.0001000017], [-...",2.456909e+09,2.926429e+05


You will calculate a new field named voter turnout using field operators in Pandas. The operations will apply to all values across the columns. 

In [112]:
# Calculate voter turnout attributes
data_2016_df['voter_turnout'] = data_2016_df['votes_total'] / data_2016_df['Total_cvap_est']
data_2016_df['voter_turnout_majparty'] = (data_2016_df['votes_dem']+data_2016_df['votes_gop']) / data_2016_df['Total_cvap_est']
data_2016_df['voter_turnout_dem'] = data_2016_df['votes_dem'] / data_2016_df['Total_cvap_est']
data_2016_df['voter_turnout_gop'] = data_2016_df['votes_gop'] / data_2016_df['Total_cvap_est']
data_2016_df['voter_turnout_other'] = data_2016_df['votes_other'] / data_2016_df['Total_cvap_est']
data_2016_df.head()

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,votes_total,votes_other,voter_share_major_party,voter_share_dem,voter_share_gop,voter_share_other,rawdiff_dem_vs_gop,rawdiff_gop_vs_dem,rawdiff_dem_vs_other,rawdiff_gop_vs_other,rawdiff_other_vs_dem,rawdiff_other_vs_gop,pctdiff_dem_vs_gop,pctdiff_gop_vs_dem,pctdiff_dem_vs_other,pctdiff_gop_vs_other,pctdiff_other_vs_dem,pctdiff_other_vs_gop,OBJECTID,GEOID,GEONAME,Total_cvap_est,SHAPE,Shape__Area,Shape__Length,voter_turnout,voter_turnout_majparty,voter_turnout_dem,voter_turnout_gop,voter_turnout_other
0,01001,2016,Autauga,Alabama,AL,President,Hillary Clinton,5936.0,Donald Trump,18172.0,24973,865.0,0.965363,0.237697,0.727666,0.034637,-12236.0,12236.0,5071.0,17307.0,-5071.0,-17307.0,-0.489969,0.489969,0.203059,0.693028,-0.203059,-0.693028,1.0,01001,"Autauga County, Alabama",40690.0,"{'rings': [[[-9619465, 3856529.0001000017], [-...",2.208654e+09,2.498864e+05,0.613738,0.592480,0.145884,0.446596,0.021258
1,01003,2016,Baldwin,Alabama,AL,President,Hillary Clinton,18458.0,Donald Trump,72883.0,95215,3874.0,0.959313,0.193856,0.765457,0.040687,-54425.0,54425.0,14584.0,69009.0,-14584.0,-69009.0,-0.571601,0.571601,0.153169,0.724770,-0.153169,-0.724770,2.0,01003,"Baldwin County, Alabama",151770.0,"{'rings': [[[-9746859, 3539643.0001000017], [-...",5.671048e+09,1.655940e+06,0.627364,0.601838,0.121618,0.480220,0.025525
2,01005,2016,Barbour,Alabama,AL,President,Hillary Clinton,4871.0,Donald Trump,5454.0,10469,144.0,0.986245,0.465278,0.520967,0.013755,-583.0,583.0,4727.0,5310.0,-4727.0,-5310.0,-0.055688,0.055688,0.451524,0.507212,-0.451524,-0.507212,3.0,01005,"Barbour County, Alabama",20375.0,"{'rings': [[[-9468394, 3771591.0001000017], [-...",3.257902e+09,3.208964e+05,0.513816,0.506748,0.239067,0.267681,0.007067
3,01007,2016,Bibb,Alabama,AL,President,Hillary Clinton,1874.0,Donald Trump,6738.0,8819,207.0,0.976528,0.212496,0.764032,0.023472,-4864.0,4864.0,1667.0,6531.0,-1667.0,-6531.0,-0.551536,0.551536,0.189024,0.740560,-0.189024,-0.740560,4.0,01007,"Bibb County, Alabama",17590.0,"{'rings': [[[-9692114, 3928124.0001000017], [-...",2.311999e+09,2.279184e+05,0.501364,0.489596,0.106538,0.383059,0.011768
4,01009,2016,Blount,Alabama,AL,President,Hillary Clinton,2156.0,Donald Trump,22859.0,25588,573.0,0.977607,0.084258,0.893348,0.022393,-20703.0,20703.0,1583.0,22286.0,-1583.0,-22286.0,-0.809090,0.809090,0.061865,0.870955,-0.061865,-0.870955,5.0,01009,"Blount County, Alabama",42430.0,"{'rings': [[[-9623907, 4063676.0001000017], [-...",2.456909e+09,2.926429e+05,0.603064,0.589559,0.050813,0.538746,0.013505


***

## Validate the data

Before continuing with other data preparation, you should confirm that the output data has been successfully created. 

First, you will validate the values for voter turnout. You will remove null values, and because these values represent a fraction (total votes divided by voting age population), you will confirm that the values range between 0 and 1.

In [113]:
# Check for null values
data_2016_df.loc[data_2016_df['voter_turnout'].isnull()]

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,votes_total,votes_other,voter_share_major_party,voter_share_dem,voter_share_gop,voter_share_other,rawdiff_dem_vs_gop,rawdiff_gop_vs_dem,rawdiff_dem_vs_other,rawdiff_gop_vs_other,rawdiff_other_vs_dem,rawdiff_other_vs_gop,pctdiff_dem_vs_gop,pctdiff_gop_vs_dem,pctdiff_dem_vs_other,pctdiff_gop_vs_other,pctdiff_other_vs_dem,pctdiff_other_vs_gop,OBJECTID,GEOID,GEONAME,Total_cvap_est,SHAPE,Shape__Area,Shape__Length,voter_turnout,voter_turnout_majparty,voter_turnout_dem,voter_turnout_gop,voter_turnout_other
67,02701,2016,District 1,Alaska,AK,President,Hillary Clinton,2573.0,Donald Trump,3180.0,6638,885.0,0.866677,0.387617,0.479060,0.133323,-607.0,607.0,1688.0,2295.0,-1688.0,-2295.0,-0.091443,0.091443,0.254293,0.345737,-0.254293,-0.345737,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,02702,2016,District 2,Alaska,AK,President,Hillary Clinton,1585.0,Donald Trump,3188.0,5492,719.0,0.869082,0.288602,0.580481,0.130918,-1603.0,1603.0,866.0,2469.0,-866.0,-2469.0,-0.291879,0.291879,0.157684,0.449563,-0.157684,-0.449563,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,02703,2016,District 3,Alaska,AK,President,Hillary Clinton,1241.0,Donald Trump,5403.0,7613,969.0,0.872718,0.163011,0.709707,0.127282,-4162.0,4162.0,272.0,4434.0,-272.0,-4434.0,-0.546696,0.546696,0.035728,0.582425,-0.035728,-0.582425,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,02704,2016,District 4,Alaska,AK,President,Hillary Clinton,4162.0,Donald Trump,4070.0,9521,1289.0,0.864615,0.437139,0.427476,0.135385,92.0,-92.0,2873.0,2781.0,-2873.0,-2781.0,0.009663,-0.009663,0.301754,0.292091,-0.301754,-0.292091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,02705,2016,District 5,Alaska,AK,President,Hillary Clinton,3187.0,Donald Trump,3683.0,7906,1036.0,0.868960,0.403112,0.465849,0.131040,-496.0,496.0,2151.0,2647.0,-2151.0,-2647.0,-0.062737,0.062737,0.272072,0.334809,-0.272072,-0.334809,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,02706,2016,District 6,Alaska,AK,President,Hillary Clinton,2536.0,Donald Trump,4929.0,8460,995.0,0.882388,0.299764,0.582624,0.117612,-2393.0,2393.0,1541.0,3934.0,-1541.0,-3934.0,-0.282861,0.282861,0.182151,0.465012,-0.182151,-0.465012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,02707,2016,District 7,Alaska,AK,President,Hillary Clinton,1510.0,Donald Trump,5935.0,8294,849.0,0.897637,0.182059,0.715578,0.102363,-4425.0,4425.0,661.0,5086.0,-661.0,-5086.0,-0.533518,0.533518,0.079696,0.613214,-0.079696,-0.613214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,02708,2016,District 8,Alaska,AK,President,Hillary Clinton,1218.0,Donald Trump,6126.0,8073,729.0,0.909699,0.150873,0.758826,0.090301,-4908.0,4908.0,489.0,5397.0,-489.0,-5397.0,-0.607952,0.607952,0.060572,0.668525,-0.060572,-0.668525,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,02709,2016,District 9,Alaska,AK,President,Hillary Clinton,1843.0,Donald Trump,6100.0,8954,1011.0,0.887090,0.205830,0.681260,0.112910,-4257.0,4257.0,832.0,5089.0,-832.0,-5089.0,-0.475430,0.475430,0.092919,0.568349,-0.092919,-0.568349,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,02710,2016,District 10,Alaska,AK,President,Hillary Clinton,1808.0,Donald Trump,6255.0,9040,977.0,0.891925,0.200000,0.691925,0.108075,-4447.0,4447.0,831.0,5278.0,-831.0,-5278.0,-0.491925,0.491925,0.091925,0.583850,-0.091925,-0.583850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [114]:
# Remove records with no voter turnout value
data_2016_df = data_2016_df.loc[data_2016_df['voter_turnout'].notnull()]

In [115]:
# Run a describe to get the distribution of voter turnout values
data_2016_df['voter_turnout'].describe()

count    3111.000000
mean        0.594528
std         0.093493
min         0.158585
25%         0.530778
50%         0.595395
75%         0.656265
max         1.121277
Name: voter_turnout, dtype: float64

The describe function indicates that there are voter turnout values over one, indicating a voter turnout above 100%. You will further investigate by querying for these records.

In [116]:
# Perform query for voter turnout above 100%
data_2016_df.loc[data_2016_df['voter_turnout'] > 1]

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,votes_total,votes_other,voter_share_major_party,voter_share_dem,voter_share_gop,voter_share_other,rawdiff_dem_vs_gop,rawdiff_gop_vs_dem,rawdiff_dem_vs_other,rawdiff_gop_vs_other,rawdiff_other_vs_dem,rawdiff_other_vs_gop,pctdiff_dem_vs_gop,pctdiff_gop_vs_dem,pctdiff_dem_vs_other,pctdiff_gop_vs_other,pctdiff_other_vs_dem,pctdiff_other_vs_gop,OBJECTID,GEOID,GEONAME,Total_cvap_est,SHAPE,Shape__Area,Shape__Length,voter_turnout,voter_turnout_majparty,voter_turnout_dem,voter_turnout_gop,voter_turnout_other
311,08111,2016,San Juan,Colorado,CO,President,Hillary Clinton,265.0,Donald Trump,215.0,506,26.0,0.948617,0.523715,0.424901,0.051383,50.0,-50.0,239.0,189.0,-239.0,-189.0,0.098814,-0.098814,0.472332,0.373518,-0.472332,-0.373518,301.0,08111,"San Juan County, Colorado",495.0,"{'rings': [[[-11964863, 4528625.000100002], [-...",1.611963e+09,209299.379233,1.022222,0.969697,0.535354,0.434343,0.052525
1816,35021,2016,Harding,New Mexico,NM,President,Hillary Clinton,156.0,Donald Trump,311.0,527,60.0,0.886148,0.296015,0.590133,0.113852,-155.0,155.0,96.0,251.0,-96.0,-251.0,-0.294118,0.294118,0.182163,0.476281,-0.182163,-0.476281,1807.0,35021,"Harding County, New Mexico",470.0,"{'rings': [[[-11578210, 4330676.000100002], [-...",8.400382e+09,492631.196575,1.121277,0.993617,0.331915,0.661702,0.127660
2684,48301,2016,Loving,Texas,TX,President,Hillary Clinton,4.0,Donald Trump,58.0,65,3.0,0.953846,0.061538,0.892308,0.046154,-54.0,54.0,1.0,55.0,-1.0,-55.0,-0.830769,0.830769,0.015385,0.846154,-0.015385,-0.846154,2674.0,48301,"Loving County, Texas",60.0,"{'rings': [[[-11502370, 3717641.0001000017], [...",2.435674e+09,254898.035389,1.083333,1.033333,0.066667,0.966667,0.050000
2689,48311,2016,McMullen,Texas,TX,President,Hillary Clinton,40.0,Donald Trump,454.0,499,5.0,0.989980,0.080160,0.909820,0.010020,-414.0,414.0,35.0,449.0,-35.0,-449.0,-0.829659,0.829659,0.070140,0.899800,-0.070140,-0.899800,2679.0,48311,"McMullen County, Texas",460.0,"{'rings': [[[-10946606, 3326438.0001000017], [...",3.882883e+09,253408.774844,1.084783,1.073913,0.086957,0.986957,0.010870


There are four counties with very low population that resulted in voter turnout values above 100%. You could remove these records from the data or do additional research to identify the source of this issue. 

***

## Update validated data

After reviewing the Census Bureau voting age population data for 2016, you determined that these counties have a low voting age population with a fairly high margin of error. This may be the reason why these counties have a voter turnout rate higher than 100%. You will recalculate the voter turnout field for these counties using the upper range of their margin of error: 
- San Juan County, Colorado: 574
- Harding County, New Mexico: 562
- Loving County, Texas: 86
- McMullen County, Texas: 566

**Note: This information was extracted from this [table](https://data.census.gov/cedsci/table?q=voting%20age%20population%202016&g=0500000US08111,35021,48301,48311&hidePreview=true&table=DP05&tid=ACSDP5Y2016.DP05&t=Age%20and%20Sex&y=2016&lastDisplayedRow=6&vintage=2016&mode=&moe=true).**

In [117]:
# Correct each county
data_2016_df.loc[data_2016_df['FIPS'] == "08111", "Total_cvap_est"] = 574
data_2016_df.loc[data_2016_df['FIPS'] == "35021", "Total_cvap_est"] = 562
data_2016_df.loc[data_2016_df['FIPS'] == "48301", "Total_cvap_est"] = 86
data_2016_df.loc[data_2016_df['FIPS'] == "48311", "Total_cvap_est"] = 566

In [118]:
# Recalculate voter turnout fields
data_2016_df['voter_turnout'] = data_2016_df['votes_total'] / data_2016_df['Total_cvap_est']
data_2016_df['voter_turnout_majparty'] = (data_2016_df['votes_dem']+data_2016_df['votes_gop']) / data_2016_df['Total_cvap_est']
data_2016_df['voter_turnout_dem'] = data_2016_df['votes_dem'] / data_2016_df['Total_cvap_est']
data_2016_df['voter_turnout_gop'] = data_2016_df['votes_gop'] / data_2016_df['Total_cvap_est']
data_2016_df['voter_turnout_other'] = data_2016_df['votes_other'] / data_2016_df['Total_cvap_est']

To confirm that this correction addressed the issue, you will again query for counties with a voter turnout value above 100%.

In [119]:
data_2016_df.loc[data_2016_df['voter_turnout'] > 1]

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,votes_total,votes_other,voter_share_major_party,voter_share_dem,voter_share_gop,voter_share_other,rawdiff_dem_vs_gop,rawdiff_gop_vs_dem,rawdiff_dem_vs_other,rawdiff_gop_vs_other,rawdiff_other_vs_dem,rawdiff_other_vs_gop,pctdiff_dem_vs_gop,pctdiff_gop_vs_dem,pctdiff_dem_vs_other,pctdiff_gop_vs_other,pctdiff_other_vs_dem,pctdiff_other_vs_gop,OBJECTID,GEOID,GEONAME,Total_cvap_est,SHAPE,Shape__Area,Shape__Length,voter_turnout,voter_turnout_majparty,voter_turnout_dem,voter_turnout_gop,voter_turnout_other


No records are returned, indicating that there are no counties with a turnout value above 100%. Well done! You have cleaned the data. Next, you will convert the dataframe to a permanent dataset called a feature class. Feature classes are stored in an ArcGIS Pro file geodatabase.

***

## Convert dataframes to feature classes

You will use the ArcGIS API for Python, imported at the beginning of this script, to export the spatially-enabled dataframe to a feature class.

**Note: Executing the following cell may take a few minutes**

In [120]:
# Create a feature class for the 2016 presidential election 
out_2016_fc_name = "county_elections_pres_2016"
out_2016_fc = data_2016_df.spatial.to_featureclass(os.path.join(fgdb, out_2016_fc_name))
out_2016_fc

'C:\\Users\\razek\\Desktop\\my_git\\Course-Summary\\Spatial Data Science The New Frontier in Analytics - ESRI\\01\\Data Engineering and Visualization\\Data Engineering and Visualization.gdb\\county_elections_pres_2016'

1. At the top of the page, click the Data Engineering map tab.

2. Drag the Data Engineering map tab to display as its own window. 

3. Review the feature class that was added to the Data Engineering map.

![DataFrameToFeatureClass](img/DataFrameToFeatureClass.PNG "Map of counties, with missing county")

**Note: The color of the data will vary every time it is added to the map.** 


In [121]:
data_2016_df

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,votes_total,votes_other,voter_share_major_party,voter_share_dem,voter_share_gop,voter_share_other,rawdiff_dem_vs_gop,rawdiff_gop_vs_dem,rawdiff_dem_vs_other,rawdiff_gop_vs_other,rawdiff_other_vs_dem,rawdiff_other_vs_gop,pctdiff_dem_vs_gop,pctdiff_gop_vs_dem,pctdiff_dem_vs_other,pctdiff_gop_vs_other,pctdiff_other_vs_dem,pctdiff_other_vs_gop,OBJECTID,GEOID,GEONAME,Total_cvap_est,SHAPE,Shape__Area,Shape__Length,voter_turnout,voter_turnout_majparty,voter_turnout_dem,voter_turnout_gop,voter_turnout_other
0,01001,2016,Autauga,Alabama,AL,President,Hillary Clinton,5936.0,Donald Trump,18172.0,24973,865.0,0.965363,0.237697,0.727666,0.034637,-12236.0,12236.0,5071.0,17307.0,-5071.0,-17307.0,-0.489969,0.489969,0.203059,0.693028,-0.203059,-0.693028,1.0,01001,"Autauga County, Alabama",40690.0,"{'rings': [[[-9619465, 3856529.0001000017], [-...",2.208654e+09,2.498864e+05,0.613738,0.592480,0.145884,0.446596,0.021258
1,01003,2016,Baldwin,Alabama,AL,President,Hillary Clinton,18458.0,Donald Trump,72883.0,95215,3874.0,0.959313,0.193856,0.765457,0.040687,-54425.0,54425.0,14584.0,69009.0,-14584.0,-69009.0,-0.571601,0.571601,0.153169,0.724770,-0.153169,-0.724770,2.0,01003,"Baldwin County, Alabama",151770.0,"{'rings': [[[-9746859, 3539643.0001000017], [-...",5.671048e+09,1.655940e+06,0.627364,0.601838,0.121618,0.480220,0.025525
2,01005,2016,Barbour,Alabama,AL,President,Hillary Clinton,4871.0,Donald Trump,5454.0,10469,144.0,0.986245,0.465278,0.520967,0.013755,-583.0,583.0,4727.0,5310.0,-4727.0,-5310.0,-0.055688,0.055688,0.451524,0.507212,-0.451524,-0.507212,3.0,01005,"Barbour County, Alabama",20375.0,"{'rings': [[[-9468394, 3771591.0001000017], [-...",3.257902e+09,3.208964e+05,0.513816,0.506748,0.239067,0.267681,0.007067
3,01007,2016,Bibb,Alabama,AL,President,Hillary Clinton,1874.0,Donald Trump,6738.0,8819,207.0,0.976528,0.212496,0.764032,0.023472,-4864.0,4864.0,1667.0,6531.0,-1667.0,-6531.0,-0.551536,0.551536,0.189024,0.740560,-0.189024,-0.740560,4.0,01007,"Bibb County, Alabama",17590.0,"{'rings': [[[-9692114, 3928124.0001000017], [-...",2.311999e+09,2.279184e+05,0.501364,0.489596,0.106538,0.383059,0.011768
4,01009,2016,Blount,Alabama,AL,President,Hillary Clinton,2156.0,Donald Trump,22859.0,25588,573.0,0.977607,0.084258,0.893348,0.022393,-20703.0,20703.0,1583.0,22286.0,-1583.0,-22286.0,-0.809090,0.809090,0.061865,0.870955,-0.061865,-0.870955,5.0,01009,"Blount County, Alabama",42430.0,"{'rings': [[[-9623907, 4063676.0001000017], [-...",2.456909e+09,2.926429e+05,0.603064,0.589559,0.050813,0.538746,0.013505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3106,56037,2016,Sweetwater,Wyoming,WY,President,Hillary Clinton,3231.0,Donald Trump,12154.0,17130,1745.0,0.898132,0.188616,0.709515,0.101868,-8923.0,8923.0,1486.0,10409.0,-1486.0,-10409.0,-0.520899,0.520899,0.086748,0.607647,-0.086748,-0.607647,3138.0,56037,"Sweetwater County, Wyoming",30565.0,"{'rings': [[[-12138709, 5200561.000100002], [-...",4.872260e+10,9.482730e+05,0.560445,0.503354,0.105709,0.397644,0.057091
3107,56039,2016,Teton,Wyoming,WY,President,Hillary Clinton,7314.0,Donald Trump,3921.0,12627,1392.0,0.889760,0.579235,0.310525,0.110240,3393.0,-3393.0,5922.0,2529.0,-5922.0,-2529.0,0.268710,-0.268710,0.468995,0.200285,-0.468995,-0.200285,3139.0,56039,"Teton County, Wyoming",16335.0,"{'rings': [[[-12251065, 5466685.000100002], [-...",2.106504e+10,6.935212e+05,0.773003,0.687787,0.447750,0.240037,0.085216
3108,56041,2016,Uinta,Wyoming,WY,President,Hillary Clinton,1202.0,Donald Trump,6154.0,8470,1114.0,0.868477,0.141913,0.726564,0.131523,-4952.0,4952.0,88.0,5040.0,-88.0,-5040.0,-0.584652,0.584652,0.010390,0.595041,-0.010390,-0.595041,3140.0,56041,"Uinta County, Wyoming",14355.0,"{'rings': [[[-12250489, 5097977.000100002], [-...",9.587254e+09,3.945276e+05,0.590038,0.512435,0.083734,0.428701,0.077604
3

***

## Correct for missing data

The feature class is missing a county in South Dakota. You will correct this issue by further exploring the data.

1. In Catalog pane, expand Databases, and then Data Engineering and Visualization.gdb.
2. Right-click Counties_2016_VotingAgePopulation and choose Add To Current Map.
3. In the Contents pane, drag Counties_2016_VotingAgePopulation under county_elections_pres_2016.
4. Open the Data Engineering tab.
5. On the map, click the missing county.

![missing county](img/missing_county_view.PNG "Pop-up window for Oglala Lakota County")

The county geometry dataset identifies the missing county as Oglala Lakota County. By searching online for this county, you determine that Oglala Lakota County changed its county name and FIPS in 2015. It was originally Shannon County with a FIPS of 46113 and is now Oglala Lakota County with a FIPS of 46102. You will search the election data for the current FIPS to try to find the missing data.

In [122]:
# Perform query for county FIPS 46102
df_out.loc[df_out['FIPS'] == '46102']

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,votes_total,votes_other,voter_share_major_party,voter_share_dem,voter_share_gop,voter_share_other,rawdiff_dem_vs_gop,rawdiff_gop_vs_dem,rawdiff_dem_vs_other,rawdiff_gop_vs_other,rawdiff_other_vs_dem,rawdiff_other_vs_gop,pctdiff_dem_vs_gop,pctdiff_gop_vs_dem,pctdiff_dem_vs_other,pctdiff_gop_vs_other,pctdiff_other_vs_dem,pctdiff_other_vs_gop


There are no records returned, which indicates that the election data does not have the correct FIPS for this county. You will check for the old FIPS value, when it was named Shannon County.

In [123]:
df_out.loc[df_out['FIPS'] == '46113']

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,votes_total,votes_other,voter_share_major_party,voter_share_dem,voter_share_gop,voter_share_other,rawdiff_dem_vs_gop,rawdiff_gop_vs_dem,rawdiff_dem_vs_other,rawdiff_gop_vs_other,rawdiff_other_vs_dem,rawdiff_other_vs_gop,pctdiff_dem_vs_gop,pctdiff_gop_vs_dem,pctdiff_dem_vs_other,pctdiff_gop_vs_other,pctdiff_other_vs_dem,pctdiff_other_vs_gop
2428,46113,2016,Oglala Lakota,South Dakota,SD,President,Hillary Clinton,2510.0,Donald Trump,241.0,2905,154.0,0.946988,0.864028,0.08296,0.053012,2269.0,-2269.0,2356.0,87.0,-2356.0,-87.0,0.781067,-0.781067,0.811015,0.029948,-0.811015,-0.029948


There is the issue! The data has the correct name (Oglala Lakota) but the wrong FIPS (46113). You will correct this data issue.

In [124]:
df_out.loc[df_out['FIPS'] == '46113', 'FIPS'] = "46102"
df_out.loc[df_out['FIPS'] == '46102']

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,votes_total,votes_other,voter_share_major_party,voter_share_dem,voter_share_gop,voter_share_other,rawdiff_dem_vs_gop,rawdiff_gop_vs_dem,rawdiff_dem_vs_other,rawdiff_gop_vs_other,rawdiff_other_vs_dem,rawdiff_other_vs_gop,pctdiff_dem_vs_gop,pctdiff_gop_vs_dem,pctdiff_dem_vs_other,pctdiff_gop_vs_other,pctdiff_other_vs_dem,pctdiff_other_vs_gop
2428,46102,2016,Oglala Lakota,South Dakota,SD,President,Hillary Clinton,2510.0,Donald Trump,241.0,2905,154.0,0.946988,0.864028,0.08296,0.053012,2269.0,-2269.0,2356.0,87.0,-2356.0,-87.0,0.781067,-0.781067,0.811015,0.029948,-0.811015,-0.029948


With the corrected FIPS value for Oglala Lakota County, you can now rejoin the geometry, recalculate the voting turnout field, and recreate the feature class. 

**Note: Executing the following cell may take a few minutes.**

In [125]:
# Join the county geometry data to the updated election data table
geo_df = pd.merge(df_out, counties_df, left_on='FIPS', right_on="GEOID", how='left')

# Create a copy of the data that only includes records from 2016
data_2016_df = geo_df.copy()
data_2016_df.query("year == '2016'", inplace=True)
data_2016_df.head()

# Correct counties with low population
data_2016_df.loc[data_2016_df['FIPS'] == "08111", "Total_cvap_est"] = 574
data_2016_df.loc[data_2016_df['FIPS'] == "35021", "Total_cvap_est"] = 562
data_2016_df.loc[data_2016_df['FIPS'] == "48301", "Total_cvap_est"] = 86
data_2016_df.loc[data_2016_df['FIPS'] == "48311", "Total_cvap_est"] = 566

# Calculate voter turnout
data_2016_df['voter_turnout'] = data_2016_df['votes_total'] / data_2016_df['Total_cvap_est']
data_2016_df['voter_turnout_majparty'] = (data_2016_df['votes_dem']+data_2016_df['votes_gop']) / data_2016_df['Total_cvap_est']
data_2016_df['voter_turnout_dem'] = data_2016_df['votes_dem'] / data_2016_df['Total_cvap_est']
data_2016_df['voter_turnout_gop'] = data_2016_df['votes_gop'] / data_2016_df['Total_cvap_est']
data_2016_df['voter_turnout_other'] = data_2016_df['votes_other'] / data_2016_df['Total_cvap_est']

# Remove records with no voter turnout value
data_2016_df = data_2016_df.loc[data_2016_df['voter_turnout'].notnull()]

You will export the dataframe to a feature class that you can visualize and analyze in ArcGIS Pro. 

**Note: Executing the following cell may take a few minutes.**

In [126]:
# Create a feature class for the 2016 election and voter turnout data
out_2016_fc_name = "county_elections_pres_2016_final"
out_2016_fc = data_2016_df.spatial.to_featureclass(os.path.join(fgdb, out_2016_fc_name))

You have prepared this data for a predictive analysis that will model voter turnout using demographic variables, such as per capita income. In the next step, you will use ArcGIS Pro to geoenrich your feature class with these demographic variables. 

Open the Perform data engineering tasks exercise PDF and refer to the Open the Enrich tool step for the remaining instructions.

***